In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import math

import nltk
nltk.download('punkt')
import random
import numpy as np

from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

from keras_preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Uncomment these and change train file path if you are running on Jupyter notebook
# import os
# notebook_path = os.path.abspath("C:/Users/amrui/Desktop/rajat/train.txt")

# Change the train file path as per your train.txt location
notebook_path = "/content/train.txt"
with open(notebook_path, 'r') as f:
    train_data = [l.strip() for l in f.readlines()]

train_data = ["<s> "+sentence.lower()+" </s>" for sentence in train_data]

In [ ]:
# rows = len(train_data)
# sz = math.floor(0.1*rows)

# train_set_idx = []
# test_set_idx = random.sample(range(rows), sz)
# test_set_idx =  np.array(test_set_idx)
# test_set_idx = np.sort(test_set_idx)

# for num in range(rows):
#     low = 0
#     high = sz-1
#     success = False
#     while low <= high:
#         mid = (low + high) // 2
#         if test_set_idx[mid] > num:
#             high = mid - 1
#         elif test_set_idx[mid] < num:
#             low = mid + 1
#         else:
#             success = True
#             break

#     if success == False:
#         train_set_idx.append(num)

# train_set_idx = np.array(train_set_idx)
# train_set_idx = np.sort(train_set_idx)

# split_train = []
# for num in train_set_idx:
#     split_train.append(train_data[num])

# split_test = []
# for num in test_set_idx:
#     split_test.append(train_data[num])

In [ ]:
train_len = 40
text_sequences = []
# count = 1
# sequences = {}

for sentence in train_data:
    tokens = sentence.split()
    
    # for i in range(len(tokens)):
    #     if tokens[i] not in sequences:
    #         sequences[tokens[i]] = count
    #         count+=1
    
    if len(tokens) > train_len:
        for i in range(train_len, len(tokens)):
            seq = tokens[i-train_len:i]
            text_sequences.append(seq)
    else:
        seq = tokens[:]
        text_sequences.append(seq)

# print(sequences['we'])
# print(sequences['are'])

In [ ]:
# print(text_sequences[1])
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
new_sequences = tokenizer.texts_to_sequences(text_sequences)
# print(new_sequences[6191])

new_sequences_size = len(new_sequences)
for i in range(new_sequences_size):
    pad_encoded = pad_sequences([new_sequences[i]], maxlen=train_len, truncating='pre')
    new_sequences[i] = pad_encoded[0]

# cnt = 0
# print(new_sequences[6191])
# for new_sequence in new_sequences:
#     if len(new_sequence) != 40:
#         print("ok")

# pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
# print(tokenizer)
# sequences2 = tokenizer.texts_to_sequences(['we', 'are', 'here', 'to'])
# print(new_sequences[1])
# print(sequences2)

In [ ]:
vocabulary_size = len(tokenizer.word_counts) + 1
n_sequences = np.empty([len(new_sequences), train_len], dtype='int32')

for i in range(len(new_sequences)):
    n_sequences[i] = new_sequences[i]

In [ ]:
train_inputs = n_sequences[:, :-1]
# print(train_inputs.shape)
seq_len = train_inputs.shape[1]
train_targets = n_sequences[:,-1]
# print(train_targets.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

In [ ]:
def model_fun():
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(256,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    return model

In [ ]:
mdl=model_fun()

# from sklearn.model_selection import GridSearchCV
# params={
#     'batch_size':[64, 32], 
#     'nb_epoch':[10, 20]
# }
# gs=GridSearchCV(mdl,params)
# # gs.fit(X,y)
# best_params=gs.best_params_
# accuracy=gs.best_score_

In [ ]:
mdl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

total_rows = train_inputs.shape[0]

# Increase the range if you are running on strong GPU
for repeat in range(2):
  print(repeat)
  for batch_start in range(0, total_rows, 1000):
      train_batch_inputs = train_inputs[batch_start:min(batch_start+1000,total_rows),:]
      train_batch_targets = train_targets[batch_start:min(batch_start+1000,total_rows)]
      train_batch_targets = to_categorical(train_batch_targets, num_classes=vocabulary_size)
      mdl.fit(train_batch_inputs,train_batch_targets,epochs=2,batch_size=32,verbose=1)

0
Epoch 1/2
32/32 [==============================] - 9s 33ms/step - loss: 7.7187 - accuracy: 0.4640
Epoch 2/2
32/32 [==============================] - 1s 30ms/step - loss: 4.1370 - accuracy: 0.4840
Epoch 1/2
32/32 [==============================] - 1s 31ms/step - loss: 5.7771 - accuracy: 0.4110
Epoch 2/2
32/32 [==============================] - 1s 30ms/step - loss: 4.7623 - accuracy: 0.4110
Epoch 1/2
32/32 [==============================] - 1s 27ms/step - loss: 5.8481 - accuracy: 0.3790
Epoch 2/2
32/32 [==============================] - 1s 23ms/step - loss: 4.8741 - accuracy: 0.3790
Epoch 1/2
32/32 [==============================] - 1s 23ms/step - loss: 5.5960 - accuracy: 0.4030
Epoch 2/2
32/32 [==============================] - 1s 23ms/step - loss: 4.6670 - accuracy: 0.4030
Epoch 1/2
32/32 [==============================] - 1s 23ms/step - loss: 5.4811 - accuracy: 0.3490
Epoch 2/2
32/32 [==============================] - 1s 23ms/step - loss: 4.7231 - accuracy: 0.3490
Epoch 1/2
32/32 [=

In [ ]:
print(mdl.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 39, 39)            1742988   
                                                                 
 lstm (LSTM)                 (None, 39, 256)           303104    
                                                                 
 dropout (Dropout)           (None, 39, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 44692)             5

In [ ]:
# import pickle

# # Saving the model in MyModel.obj file
# filehandler = open("MyModel.obj","wb")
# pickle.dump(mdl,filehandler)
# filehandler.close()

# print("The model has been saved in MyModel.obj file")

In [ ]:
import pickle

# Saving the model in MyModel.obj file
filehandler = open("/content/drive/MyDrive/MyModel/MyModel.obj","wb")
pickle.dump(mdl,filehandler)
filehandler.close()

print("The model has been saved in MyModel.obj file")

The model has been saved in MyModel.obj file


In [ ]:
# Saving the tokenizer in MyTokenizer.obj file
filehandler = open("/content/drive/MyDrive/MyModel/MyTokenizer.obj", "wb")
pickle.dump(tokenizer, filehandler)
filehandler.close()

In [ ]:
# Note that from here both the model and the tokenizer is saved on the drive